# 2.5 自动微分
- 深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导
- 实际中，根据我们设计的模型，系统会构建一个计算图（computational graph）， 来跟踪计算是哪些数据通过哪些操作组合起来产生输出
## 2.5.1 例子
对函数y=2x<sup>T</sup>x关于列向量x求导

In [1]:
import torch
x=torch.arange(4.0) # create a row vector
x

tensor([0., 1., 2., 3.])

In [2]:
"""
在我们计算y关于x的梯度之前，我们需要一个地方来存储梯度。

重要的是，我们不会在每次对一个参数求导时都分配新的内存。 因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。

注意，一个标量函数关于向量x的梯度是向量，并且与x具有相同的形状。
"""

x.requires_grad=True  # 等价于 x = torch.arange(4.0,requires_grad=True)，需要计算梯度
x.grad  # 默认值是None

# torch.dot(x,x)结果为标量14(0+1+4+9)
y = 2 * torch.dot(x,x)  # y = 2 * (x · x) = 2 * sum(x_i^2)
y


tensor(28., grad_fn=<MulBackward0>)

In [3]:
"""
通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。
"""
y.backward()
x.grad


tensor([ 0.,  4.,  8., 12.])

In [4]:
"""
验证梯度是否计算正确
"""
x.grad==4*x

tensor([True, True, True, True])

In [5]:
"""
在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
"""
x.grad.zero_()
y = x.sum()
y.backward()
y,x.grad

(tensor(6., grad_fn=<SumBackward0>), tensor([1., 1., 1., 1.]))

# 2.5.2 标量变量的反向传播

---

## 问题复述：

我们有以下 PyTorch 代码：

```python
import torch

x = torch.arange(4.0, requires_grad=True)  # x = [0., 1., 2., 3.]
y = 2 * torch.dot(x, x)                    # y = 2 * (x · x) = 2 * sum(x_i^2)
```

我们的问题是：**如何计算 y 关于 x 的梯度 ∂y/∂x\_i？**

---

## Step 1：数学上先推导目标

设：

$$
x = \begin{bmatrix} x_0 \\ x_1 \\ x_2 \\ x_3 \end{bmatrix}
$$

那么：

$$
y = 2 \cdot (x_0^2 + x_1^2 + x_2^2 + x_3^2)
= 2 \sum_{i=0}^{3} x_i^2
$$

对每个分量求导：

$$
\frac{\partial y}{\partial x_i} = 2 \cdot \frac{d}{dx_i}(x_i^2) = 2 \cdot 2x_i = 4x_i
$$

所以：

$$
\nabla_x y = \left[ 4x_0, 4x_1, 4x_2, 4x_3 \right]
= 4 \cdot x
$$

---

## Step 2：在 PyTorch 中验证这个计算

```python
import torch

# 定义变量
x = torch.arange(4.0, requires_grad=True)  # x = tensor([0., 1., 2., 3.], requires_grad=True)

# 构造函数 y = 2 * dot(x, x) = 2 * sum(x_i^2)
y = 2 * torch.dot(x, x)  # y = 2*(0^2 + 1^2 + 2^2 + 3^2) = 2 * 14 = 28

# 反向传播计算梯度
y.backward()

# 查看梯度
print(x.grad)  # 输出：tensor([ 0.,  4.,  8., 12.])
```

这和我们数学计算的 $4 \cdot x = [0., 4., 8., 12.]$ 完全一致 ✅

---

## Step 3：逐行解释代码及工作原理

| 代码                   | 含义                                                  |
| -------------------- | --------------------------------------------------- |
| `torch.arange(4.0)`  | 构造一个一维张量 `[0., 1., 2., 3.]`                         |
| `requires_grad=True` | 告诉 PyTorch 跟踪这个变量的计算图，允许求导                          |
| `torch.dot(x,x)`     | 执行内积（点积）：即 $\sum x_i^2$                             |
| `2 * torch.dot(x,x)` | 构造函数 $y = 2 \sum x_i^2$                             |
| `y.backward()`       | 启动自动反向传播，从 y 开始逐层反向计算每个变量的梯度                        |
| `x.grad`             | 读取 x 的每个元素对 y 的梯度 $\frac{\partial y}{\partial x_i}$ |

---

## Step 4：验证 autograd 做了什么

`y.backward()` 实际上在做：

* 先从最终的标量 `y` 反向传播；
* 追踪图中所有对 `x` 的操作（此处是 `dot` 和乘法）；
* 对每一条计算路径应用**链式法则（chain rule）**；
* 最终得到 `x.grad[i] = ∂y/∂x[i]`

---

## 小结（适合笔记整理）

> 在 PyTorch 中，自动微分框架 `autograd` 可以通过 `.backward()` 方法对标量函数自动求梯度。当构造函数 $y = 2 \cdot \text{dot}(x,x)$ 时，实际数学含义为：
>
> $$
> y = 2 \sum x_i^2 \quad \Rightarrow \quad \frac{\partial y}{\partial x_i} = 4x_i
> $$
>
> 使用 `requires_grad=True` 追踪张量后，调用 `y.backward()` 可以自动得到每个输入变量的梯度，并保存在 `x.grad` 中，精确符合手动推导结果。




## 2.5.3 非标量变量的反向传播

##  定义

在深度学习中，**反向传播（backpropagation）**的目标是计算函数的**梯度（gradient）**，即输出对输入变量的偏导数。

当你使用 `.backward()` 方法时：

* **标量变量**（如：损失函数） → 可以**直接调用 `y.backward()`**；
* **非标量变量**（如：向量、矩阵） → 必须**提供 `gradient=` 参数**，即对该张量施加的外部“权重”导数。

---

##  本质：非标量变量没有“单一方向”的梯度

### 标量情况（容易）：

例如：

```python
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.dot(x, x)   # y 是标量
y.backward()
```

$$
\frac{\partial y}{\partial x} = \left[ \frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, \dots \right]
$$

这是非常标准的“从一个标量出发”的反向传播，返回的是梯度向量。

---

### 非标量情况（难点）：

```python
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * 2             # y 是向量 [2.0, 4.0, 6.0]
y.backward()          # ❌ 会报错
```

因为：

* `y` 是一个**向量**，包含多个输出；
* 那么，**“哪个方向”来传播导数呢？**
* **你没有定义一个标量损失**，所以 PyTorch 不知道该如何聚合这些输出。

---

##  数学解释：雅可比矩阵与链式法则

设：

* $\mathbf{x} \in \mathbb{R}^n$
* $\mathbf{y} = f(\mathbf{x}) \in \mathbb{R}^m$

那么：

* 输出对输入的梯度是一个 **雅可比矩阵（Jacobian）**：

$$
J = \frac{\partial \mathbf{y}}{\partial \mathbf{x}} \in \mathbb{R}^{m \times n}
$$

当你调用 `.backward(v)` 时，其实是执行了链式法则中的：

$$
\frac{dL}{dx} = v^\top \cdot \frac{\partial y}{\partial x}
$$

其中 $v \in \mathbb{R}^{m}$ 是你提供的 `gradient=` 参数，相当于外部损失函数对 $y$ 的导数。

---

##  PyTorch 中的实际操作示例

###  示例 1：非标量输出 + `.backward(gradient=...)`

```python
import torch

x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * 2                        # y = [2.0, 4.0, 6.0]

v = torch.tensor([1.0, 1.0, 1.0])  # 外部“损失对 y 的导数”： ∂L/∂y
y.backward(gradient=v)            # 显式指定方向

print(x.grad)  # 输出：tensor([2., 2., 2.])，因为 dy/dx = 2
```

你告诉系统：假设外部损失函数是 $L = \sum y_i$，那么我们对每个 $y_i$ 的梯度是 1，这就能回传到 $x$ 上。

---

###  示例 2：更复杂的例子

```python
x = torch.tensor([1., 2.], requires_grad=True)
y = torch.stack([x[0]**2, x[1]**3])     # y ∈ ℝ²，y = [x₀², x₁³]

v = torch.tensor([1., 1.])             # ∂L/∂y 假设都是 1

y.backward(v)

# x.grad[0] = ∂L/∂x₀ = ∂L/∂y₀ · ∂y₀/∂x₀ = 1 · 2x₀ = 2
# x.grad[1] = ∂L/∂x₁ = ∂L/∂y₁ · ∂y₁/∂x₁ = 1 · 3x₁² = 12

print(x.grad)  # 输出：tensor([2., 12.])
```

---

##  常见误区

| 误区                    | 正确理解                    |
| --------------------- | ----------------------- |
| “非标量也能直接 backward”    |  只有标量才能直接调用 `.backward()` |
| “自动微分总是能自动处理”         |  非标量必须提供 `gradient=`    |
| “gradient 参数必须是 1 向量” |  不一定，可以是任何方向上的导数向量      |

---

##  总结（建议笔记整理）

> 在 PyTorch 中，只有标量张量才能直接调用 `.backward()` 来自动反向传播。
> 当输出变量是非标量（如向量或矩阵）时，系统无法确定从哪个方向传播导数，必须手动提供一个与输出形状相同的权重张量 `gradient=`。
> 该张量表示“损失函数对输出变量的导数”，从而触发链式法则中雅可比矩阵乘法，完成反向传播。




In [6]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
y.sum().backward()  # 等价于y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

## 2.5.3 分离计算
- 将某些计算移动到记录的计算图之外。

In [7]:
"""
下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。
"""
x.grad.zero_()  # 清0梯度
y = x * x
u = y.detach()  # u不再是关于x的函数，而是一个常数
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [8]:
"""
由于记录了y的计算结果，我们可以随后在y上调用反向传播， 得到y=x*x关于的x的导数，即2*x。
"""
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## 2.5.4. Python控制流的梯度计算
- 使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。

In [19]:
"""
在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。
"""
def f(a):
    b = a*2
    # print(b)
    while b.norm()<1000:
        b = b*2
        # print(b)
    if b.sum() > 0:
        c=b
    else:
        c = 100*b
    return c

# 计算梯度
a=torch.randn(size=(),requires_grad=True)  # 定义a为一个随机数
d=f(a)
d.backward()
"""
我们现在可以分析上面定义的f函数。 请注意，它在其输入a中是分段线性的。 换言之，对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输入a。 因此，我们可以用d/a验证梯度是否正确。
"""
a,a.grad==d/a

(tensor(-9.0155e-05, requires_grad=True), tensor(False))

## 2.5.5 小结
- 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上。然后我们记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

# Q1 哪些是标量损失函数？哪些是非标量损失函数？他们是怎么工作的？

## 一、什么是“标量损失函数”？

### 定义：

> 标量损失函数是指：模型输出通过损失函数后，返回一个**单个实数值（scalar）**，表示整个样本或样本批次的“预测误差”。

### 常见例子：

| 损失函数                                 | 输出形状     | 含义      |
| ------------------------------------ | -------- | ------- |
| `torch.nn.MSELoss(reduction='mean')` | `scalar` | 平均平方误差  |
| `torch.nn.CrossEntropyLoss()`        | `scalar` | 交叉熵损失   |
| `torch.nn.NLLLoss()`                 | `scalar` | 负对数似然损失 |
| `torch.nn.BCELoss()`                 | `scalar` | 二分类交叉熵  |

### 特点：

* 可直接调用 `.backward()`：

  ```python
  loss = criterion(pred, target)
  loss.backward()  # ✔️ 因为 loss 是 scalar
  ```
* 训练时几乎总是使用标量损失函数；
* 是深度学习训练的**默认模式**。

---

## 二、什么是“非标量损失函数”？

### 定义：

> 非标量损失函数是指：**每个样本**或**每个输出位置**都有自己的损失值，损失函数输出的是一个**向量或矩阵（tensor）**。

例如：

* 返回 shape = `[batch_size]`：每个样本一个损失；
* 返回 shape = `[batch_size, C, H, W]`：用于像素级别监督任务。


### 常见例子：

| 损失函数                                     | 设置          | 输出             | 场景        |
| ---------------------------------------- | ----------- | -------------- | --------- |
| `torch.nn.MSELoss(reduction='none')`     | 无 reduction | `[B, C, H, W]` | 图像逐点监督    |
| `F.cross_entropy(..., reduction='none')` | 无 reduction | `[B]`          | 每个样本自己的损失 |
| 自定义 loss（如 contrastive loss）             | 手动构造        | 向量或矩阵          | 表示多个比较对   |

### 特点：

* **不能直接 `.backward()`**，因为非标量不能自定义传播方向；
* **必须先聚合为标量（如 `.mean()` 或 `.sum()`）再传播**：

```python
loss = F.cross_entropy(pred, target, reduction='none')  # [B]
loss = loss.mean()  # scalar
loss.backward()
```

或者：

```python
loss = custom_vector_loss(pred, target)  # [B]
loss.backward(torch.ones_like(loss))  # 手动提供导数权重向量
```

---

## 三、两种损失函数在训练中的比较

| 特性                  | 标量损失（scalar loss）    | 非标量损失（tensor loss） |
| ------------------- | -------------------- | ------------------ |
| 输出形状                | `torch.Size([])`（0维） | 多维张量               |
| 是否能直接 `.backward()` | ✔️ 可以                | ❌ 不可以              |
| 是否常用于训练             | ✅ 是                  | ⛔ 通常先聚合            |
| 使用场景                | 分类、回归、单任务训练          | 多任务、注意力监督、对比学习     |
| 优势                  | 简洁、稳定、自动传播           | 灵活、细粒度监督、多尺度损失     |
| 缺点                  | 不可细粒度控制              | 需手动聚合或手动传播方向       |

---

## 四、实例对比：MSELoss 标量 vs 非标量

### 标量形式（默认）：

```python
criterion = torch.nn.MSELoss()  # 默认 reduction='mean'
y_pred = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y_true = torch.tensor([1.0, 2.0, 1.0])
loss = criterion(y_pred, y_true)  # loss 是 scalar
loss.backward()  # ✅ 自动传播
```

### 非标量形式（逐元素）：

```python
criterion = torch.nn.MSELoss(reduction='none')
loss = criterion(y_pred, y_true)  # loss 是 [3] 向量
print(loss)  # tensor([0.0, 0.0, 4.0])

# loss.backward()  ❌ 报错，不能直接反向传播

loss.mean().backward()  # ✅ 先聚合为标量
```

---

## 五、实际应用中的非标量 loss

### 多任务学习：

* 每个任务对应一个分量：`loss = [loss_cls, loss_reg, loss_mask]`
* 最终手动聚合成标量（加权求和）

### 语义分割（pixel-wise loss）：

* `loss.shape = [B, C, H, W]`，每个像素都有损失；
* 通常 `loss.mean()` 聚合后再 `.backward()`。

### 对比学习、距离学习：

* 每对样本有一个损失值：`loss.shape = [B, B]`
* 聚合为 `loss.mean()` 或手动加权传播

---

## 六、总结（适合记笔记）

> * **标量损失函数（scalar loss function）**：输出单个实数，可直接 `.backward()`，是训练神经网络时的默认形式（如 MSE、交叉熵）。
> * **非标量损失函数（non-scalar loss function）**：输出多维张量，如每个样本、每个像素的损失，需要聚合为标量后才能反向传播。
> * 在 PyTorch 中，非标量损失必须通过 `.mean()` 或手动提供 `.backward(gradient=...)` 来引导反向传播。



